In [7]:
from dash import Dash, html, dcc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob


In [10]:
def read_files(folder_path, file_extension):
    # Create a file path pattern using glob
    pattern = os.path.join(folder_path, f"*.{file_extension}")
    filename = []

    # Use glob to get a list of files matching the pattern
    files = glob.glob(pattern)
    for file in files:
        filenames.append(file.split("\\")[1])

    # Return the count of matching files
    return filenames

# Example usage
folder_path = "C:/Users/user/Documents/GitHub/Automotive_Diagnostics/OBD-II-Dataset"
file_extension = "csv"

data_files = read_files(folder_path, file_extension)
print(data_files)

NameError: name 'filenames' is not defined

In [5]:
app = Dash(__name__)

app.layout = html.Div([
    # dcc.h1("Automotive Dashboard"),
    "Select a data to visualise:!",
    dcc.Dropdown(
        options=["Red", "Green", "Blue"],
        id="color-input",
        value="Red"
    ),
])

if __name__ == "__main__":
    app.run_server(mode="inline", port=8051)